In [ ]:
pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-8o_gbd5d
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-8o_gbd5d
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-

In [ ]:
!pip install transformers
!pip install whisper
!pip install pydub
!sudo apt install portaudio19-dev
!pip install pyaudio
!pip install -qq https://github.com/pyannote/pyannote-audio/archive/refs/heads/develop.zip
!pip install --upgrade whisper
!pip install speechbrain=0.5.16

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for whisper: filename=whisper-1.1.10-py3-none-any.whl size=41120 sha256=d8e3548b05d5814f717d480aaa481347b7a6f39f78a12026ec7018c05ce6a5e3
  Stored in directory: /root/.cache/pip/wheels/aa/7c/1d/015619716e2facae6631312503baf3c3220e6a9a3508cb14b6
Successfully built whisper
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libportaudio2 libportaudiocpp0
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 188 kB of archives.
After this operation, 927 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http:

In [ ]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import whisper
import os
import re
from datetime import datetime, timedelta
from pydub import AudioSegment
from pydub.playback import play
import pyaudio
import subprocess
import torch
import pyannote.audio
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding
from pyannote.audio import Audio
from pyannote.core import Segment
import wave
import contextlib
from sklearn.cluster import AgglomerativeClustering
import numpy as np
import soundfile as sf
import random

Found existing installation: speechbrain 1.0.0
Uninstalling speechbrain-1.0.0:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/speechbrain-1.0.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/speechbrain/*
    /usr/local/lib/python3.10/dist-packages/tests/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.10/dist-packages/tests/test_optimizer.py
    /usr/local/lib/python3.10/dist-packages/tests/test_pipeline.py
    /usr/local/lib/python3.10/dist-packages/tests/utils.py
Proceed (Y/n)? Y
  Successfully uninstalled speechbrain-1.0.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 630.6/630.6 kB 12.9 MB/s eta 0:00:00


In [ ]:
def speaker_diarization(path, number, language, size):
    path = path

    num_speakers = 2  # @param {type:"integer"}
    language = "English"  # @param ['any', 'English']
    model_size = "base"  # @param ['tiny', 'base', 'small', 'medium', 'large']

    model_name = model_size
    if language == 'English' and model_size != 'medium':
        model_name += '.en'

    embedding_model = PretrainedSpeakerEmbedding("speechbrain/spkrec-ecapa-voxceleb", device=torch.device("cpu"))

    tokenizer = AutoTokenizer.from_pretrained("pszemraj/led-base-book-summary")
    audio = AudioSegment.from_file(path, format="wav")
    channels = audio.channels
    print("Number of channels:", channels)

    # If the audio has more than one channel, convert it to a single-channel audio
    if channels > 1:
        audio = audio.set_channels(1)
        audio.export('mono_audio.wav', format="wav")
        path = 'mono_audio.wav'

    # List of first names and last names
    first_names = ["Emma", "Liam", "Ava", "Noah", "Sophia", "William", "Isabella", "James", "Mia", "Benjamin",
                   "Charlotte", "Oliver", "Amelia", "Evelyn", "Henry", "Harper", "Ella", "Alexander", "Abigail", "Michael"]
    last_names = ["Smith", "Johnson", "Brown", "Garcia", "Miller", "Davis", "Rodriguez", "Martinez", "Hernandez", "Lopez",
                  "Gonzalez", "Perez", "Taylor", "Anderson", "Wilson", "Jackson", "White", "Harris", "Martin", "Thompson"]

    # Function to generate a random name
    def generate_name():
        first_name = random.choice(first_names)
        last_name = random.choice(last_names)
        return f"{first_name} {last_name}"

    if path[-3:] != 'wav':
        subprocess.call(['ffmpeg', '-i', path, 'audio.wav', '-y'])
        path = 'audio.wav'

    model = whisper.load_model(model_size)

    result = model.transcribe(path)
    segments = result["segments"]

    with contextlib.closing(wave.open(path, 'r')) as f:
        frames = f.getnframes()
        rate = f.getframerate()
        duration = frames / float(rate)

    audio = Audio()

    def segment_embedding(segment):
        start = segment["start"]
        # Whisper overshoots the end timestamp in the last segment
        end = min(duration, segment["end"])
        clip = Segment(start, end)
        waveform, samplerate = audio.crop(path, clip)
        return embedding_model(waveform[None])

    embeddings = np.zeros(shape=(len(segments), 192))
    for i, segment in enumerate(segments):
        embeddings[i] = segment_embedding(segment)

    embeddings = np.nan_to_num(embeddings)

    clustering = AgglomerativeClustering(num_speakers).fit(embeddings)
    labels = clustering.labels_
    for i in range(len(segments)):
        segments[i]["speaker"] = 'SPEAKER ' + str(labels[i] + 1)

    def time(secs):
        return timedelta(seconds=round(secs))

    f = open("transcript.txt", "w", encoding='utf-8')
    speakers = []
    distinct_speakers = []
    for (i, segment) in enumerate(segments):
        if i == 0 or segments[i - 1]["speaker"] != segment["speaker"]:
            f.write("\n" + segment["speaker"] + ' ' + str(time(segment["start"])) + '\n')
            temp1 = time(segment["start"])
            temp2 = time(segment["end"])
            speakers.append(segment["speaker"])
        f.write(segment["text"][1:] + ' ')
        temp = set(speakers)
        for x in temp:
            if x not in distinct_speakers:
                distinct_speakers.append(segment["speaker"])
                distinct_speakers.append(segment["start"])
                distinct_speakers.append(segment["end"])

    print("speakers are ::", speakers)
    print("ds are ::", distinct_speakers)
    f.close()

    print(open("transcript.txt", 'r', encoding='utf-8').read())

    print()
    # Regex pattern to match the speaker and dialogue
    pattern = re.compile(r'SPEAKER (\d+) 0:(\d\d):(\d\d)\n(.*)')
    # Open the file for reading
    with open("transcript.txt", "r", encoding='utf-8') as f:
        content = f.read()

    # Dictionary to store the dialogues for each speaker
    dialogues = {}

    # Loop through all the matches in the content
    for match in re.finditer(pattern, content):
        speaker = "SPEAKER " + str(match.group(1))
        dialogue = match.group(4)

        # If the speaker is not already in the dictionary, add it and start a list
        if speaker not in dialogues:
            dialogues[speaker] = []

        # Add the dialogue to the list for this speaker
        dialogues[speaker].append(dialogue)

    data, samplerate = sf.read(path)
    print("the sample rate is :", samplerate)
    audio_file = AudioSegment.from_file(path, format="wav")
    speaker_names = []
    val = len(distinct_speakers) // 3

    for i in range(val):
        j = 1 + i * 3
        start_time_str = str(distinct_speakers[j])
        end_time_str = str(distinct_speakers[j + 1])
        start_time = float(start_time_str)
        end_time = float(end_time_str)
        segment = audio_file[start_time * 1000:end_time * 1000]

        try:
            p = pyaudio.PyAudio()
            device_index = p.get_default_output_device_info()['index']
            stream = p.open(output_device_index=device_index, format=pyaudio.paInt16, channels=2, rate=35000, output=True)
            stream.write(segment.raw_data)
            stream.stop_stream()
            stream.close()
            p.terminate()
        except OSError as e:
            print(f"Skipping audio playback: {e}")

        name = generate_name()
        speaker_names.append(name)
        export_file = name + ".mp3"
        segment.export(export_file, format="mp3")

    for i in range(val):
        distinct_speakers[i * 3] = speaker_names[i]

    print(distinct_speakers)
    print(speaker_names)

    i = 0

    for speaker, speaker_dialogues in dialogues.items():
        with open(speaker_names[i] + ".txt", "w", encoding='utf-8') as f:
            f.write(speaker_names[i] + ":" + "\n")
            f.write("\n".join(speaker_dialogues))
        i = i + 1

    with open("transcript.txt", "r", encoding='utf-8') as f:
        text = f.read()
    model = AutoModelForSeq2SeqLM.from_pretrained("pszemraj/led-base-book-summary")
    input_ids = tokenizer.encode(text, return_tensors="pt")
    summary_ids = model.generate(input_ids, num_beams=4, length_penalty=2.0, max_length=142, min_length=56,
                                 early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    with open("summary.txt", "w", encoding='utf-8') as f:
        for sentence in summary:
            f.write(str(sentence))

    for m in range(len(speaker_names)):
        if (os.path.exists(speaker_names[m] + ".txt")):
            with open(speaker_names[m] + ".txt", "r", encoding='utf-8') as f:
                text = f.read()
            input_ids = tokenizer.encode(text, return_tensors="pt")
            summary_ids = model.generate(input_ids, num_beams=4, length_penalty=2.0, max_length=142, min_length=56,
                                         early_stopping=True)
            summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
            with open(speaker_names[m] + "summary" + ".txt", "w", encoding='utf-8') as f:
                for sentence in summary:
                    f.write(str(sentence))
        else:
            break

    individual_summary_files_store = []
    individual_script_files_store = []
    actions_files_store = []

    for name in speaker_names:
        actions_files_store.append(name + "actions.txt")
        individual_script_files_store.append(name + ".txt")
        individual_summary_files_store.append(name + "summary.txt")

    transcript_file = 'transcript.txt'
    summary_file = 'summary.txt'
    actions_files = actions_files_store
    individual_script_files = individual_script_files_store
    individual_summary_files = individual_summary_files_store

    def generate_json(transcript_file, summary_file, actions_files, individual_script_files, individual_summary_files):

      with open("output.json", 'r') as json_file:
        data = json.load(json_file)
        final_json = json.dumps(data, indent=4)

      return final_json

speaker_diarization('/content/myaudio.wav', 2, 'English', 'base')

Number of channels: 1
speakers are :: ['SPEAKER 1', 'SPEAKER 2', 'SPEAKER 1', 'SPEAKER 2', 'SPEAKER 1', 'SPEAKER 2', 'SPEAKER 1']
ds are :: ['SPEAKER 1', 0.0, 5.0, 'SPEAKER 2', 5.0, 7.0]

SPEAKER 1 0:00:00
So you hear about recent updates from Reserve Bank of India regarding UPI payments? 
SPEAKER 2 0:00:05
No, what's new? 
SPEAKER 1 0:00:07
Well, the RBI has increased the limit for automatic payments through UPI to rupees 1 lakh per transaction from the earlier rupees 15,000, especially for certain categories like mutual fund subscriptions. 
SPEAKER 2 0:00:23
Wow, that's a significant jump. It will definitely make things more convenient for many, especially those dealing with higher value transaction. 
SPEAKER 1 0:00:33
Absolutely. And there's another interesting bit. The relaxation and additional factor of authentication for subsequent recurring transactions up to rupees 15,000 on cards, repair instruments and UPI remains in place. 
SPEAKER 2 0:00:51
There's no need to know. It will 

Input ids are automatically padded from 129 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 68 to 1024 to be a multiple of `config.attention_window`: 1024
